# Running TIMES Models using Python Notebooks

This is a demonstration of how to use Python (Jupyter) notebooks to define, run, modify, and analyze TIMES model scenarios.

Using `xl2times`'s new `TimesModel` API, one can run model scenarios in an interactive, reproducible, self-documenting, and programmable way.

## Setup the Demo on Colab

This section is not neccessary if you have installed `xl2times` and the Demo models on your local machine.

In [2]:
%pip install gdx2py gamspy-base git+https://github.com/etsap-TIMES/xl2times.git

  Cloning https://github.com/etsap-TIMES/xl2times.git to /tmp/pip-req-build-0ql_n06v
  Running command git clone --filter=blob:none --quiet https://github.com/etsap-TIMES/xl2times.git /tmp/pip-req-build-0ql_n06v
  Resolved https://github.com/etsap-TIMES/xl2times.git to commit c6e062d74bea0928aa6b0c23ff3b292dbecad0f9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.7 MB/s eta 0:00:00
  Created wheel for xl2times: filename=xl2times-0.2.2-py3-none-any.whl size=80618 sha256=fd8715b0da2e6469a2050b0e93447fe23b171cf862de113832d7dc3a94bfec08
  Stored in directory: /tmp/pip-ephem-wheel-cache-gaifqa2a/wheels/d4/b5/63/8fcb20cf732167e710e5117b2dc5eb8154d9972bbf164fb9c1
Successfully built xl2times


In [ ]:
%%bash
# Download the Demo 3 model
curl 'https://drive.usercontent.google.com/download?id=10zeI0xDk4g_I1e5xFwiC7e74Z21uYgWN&export=download&authuser=0&confirm=t' -o DemoS_003.zip
unzip DemoS_003.zip
# Clone the TIMES source code
git clone --filter=blob:none https://github.com/etsap-TIMES/TIMES_model TIMES_model
pushd TIMES_model/
git checkout b488fb07f0899ee8b7e710c230b1a9414fa06f7d
popd

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  226k  100  226k    0     0  85880      0  0:00:02  0:00:02 --:--:-- 85902


## Run a model scenario

In [3]:
from os import path

import gdx2py
from gamspy_base import directory

import xl2times.main as xt
from xl2times.utils import run_gams, setup_logger

In [25]:
setup_logger(level=0)
input_dir = "/content/DemoS_003/"
output_dir = "/content/DemoS_003-all-out/"
times_dir = "/content/TIMES_model/"

model, config = xt.read_xl([input_dir], regions="", include_dummy_imports=True)

/usr/local/lib/python3.11/dist-packages/xl2times/transforms.py:1276: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'EOH' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[
Applying transformations from ~TFM_UPD in SYSSETTINGS: 100%|██████████| 2/2 [00:00<00:00, 97.03it/s]


In [26]:
# Inspect some key parameters in the solution
def inspect_solution():
    with gdx2py.GdxFile(path.join(output_dir, "scenario.gdx"), gams_dir=directory) as g:
        print(g["REG_OBJ"].values())
        print(g["PAR_COMPRDL"].values())
        print(g["EQE_COMPRD"])

In [29]:
print(model.attributes.query('attribute == "COM_IE"')["value"])
tables = xt.to_tables(config, model)
xt.write_dd_files(tables, config, output_dir)

run_gams(times_dir, output_dir)

inspect_solution()

169    0.9
Name: value, dtype: object
2025-06-07 10:16:51.921 |  SUCCESS : Excel files successfully converted to DD and written to /content/DemoS_003-all-out/
2025-06-07 10:16:52.320 |  SUCCESS : Ran GAMS successfully on /content/DemoS_003-all-out/:
*** Status: Normal completion--- Job scenario.run Stop 06/07/25 10:16:52 elapsed 0:00:00.374
dict_values([3195557.668833001])
dict_values([])
None


## Modify some assumptions and re-run the scenario

One can think of extending this to programmatically run multiple scenarios, automatically varying a parameter from a set of options, etc.

In [30]:
# Modify COM_IE and re-run the scenario
com_ie = model.attributes.query('attribute == "COM_IE"')
model.attributes.loc[com_ie.index, "value"] = 0.95
print(model.attributes.query('attribute == "COM_IE"')["value"])

tables = xt.to_tables(config, model)
xt.write_dd_files(tables, config, output_dir)

run_gams(times_dir, output_dir)

inspect_solution()

169    0.95
Name: value, dtype: object
2025-06-07 10:17:51.023 |  SUCCESS : Excel files successfully converted to DD and written to /content/DemoS_003-all-out/
2025-06-07 10:17:51.392 |  SUCCESS : Ran GAMS successfully on /content/DemoS_003-all-out/:
*** Status: Normal completion--- Job scenario.run Stop 06/07/25 10:17:51 elapsed 0:00:00.347
dict_values([3095381.6789671676])
dict_values([])
None
